<a href="https://colab.research.google.com/github/RotcehOdraude/Criptografia-UNAM-2021-1/blob/main/Hill.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cifrado Hill

## Definición de funciones

In [ ]:
#En caso de no encontrar el modulo array-to-latex, instalar con:
!pip install array-to-latex

In [2]:
import array_to_latex as a2l
from IPython.display import Math, display, Markdown
from numpy import array,zeros,size,matrix,around
from numpy.linalg import det

def es_multiplo(llave):
  ''' devuelve verdadero o falso dependiendo de si el numero de caracteres de 
  la llave es multiplo de la dimension de la matriz'''
  if len(llave) % 3 != 0:
    #print("No es multiplo")
    return False
  else:
    #print("Es multiplo")
    return True
def hacer_matriz(llave,dimension):
  ''' Esta funcion transforma una cadena de caracteres en una matriz'''
  matriz = []
  for i in zip(range(0,len(llave)+1,dimension),range(dimension-1,len(llave),dimension)):
    matriz.append(list(llave[i[0]:i[1]+1]))
  return matriz
    
def rellenar(llave,dimension):
  ''' Esta funcion rellena de X(equis) la llave en caso de no ser multiplo '''
  numero_de_caracteres = dimension*dimension
  numero_relleno = numero_de_caracteres-len(llave)
  a = list(llave)

  if numero_relleno < 0:
    while len(a)%dimension != 0:
      a.append('x')
    return "".join(a)

  for i in range(numero_relleno):
    a.append('x')
  return "".join(a)

def transformar_caracteres(matriz_caracteres,equivalencias):
  ''' Esta funcion transforma una matriz de caracteres a una matriz de numeros
  dada una tabla de equivalencias'''
  numero_renglones = len(matriz_caracteres)
  numero_columnas = len(matriz_caracteres[0])

  for i in range(numero_renglones):
    for j in range(numero_columnas):
      caracter = matriz_caracteres[i][j]
      try:
        matriz_caracteres[i][j] = equivalencias[caracter]
      except:
        print(f"La llave [{caracter}] no se encuentra en el diccionario")
        caracter = caracter % modulo
        print(f"Probando nueva llave... = {caracter}")
        matriz_caracteres[i][j] = equivalencias[caracter]
  return matriz_caracteres

def printl(matriz_lista):
  ''' Esta funcion imprime una matriz en forma de latex'''
  A = array(matriz_lista)
  latex_raw = a2l.to_ltx(A,arraytype="pmatrix",print_out=False,frmt='{:n}')
  display(Math(latex_raw))

def printm(codigo_markdown):
  ''' Esta funcion imprime en forma de markdown'''
  display(Markdown(codigo_markdown))
    
def inverso_multiplicativo(numero, modulo):
  ''' Esta funcion calcula el inverso multiplicativo de un numero dado un modulo'''
  lista_una_de_la_tarde = [(modulo + 1)+(modulo * k) for k in range(modulo)]
  for elemento in lista_una_de_la_tarde:
    if elemento % numero == 0:
      inverso = (elemento/numero) % modulo
      return int(inverso)
  print(f"El {numero} no tiene inverso multiplicativo modulo {modulo}")
  return None

def adjunta(matriz):
  ''' Esta funcion calcula la adjunta de la matriz''' 
  A=matrix(matriz)
  MC=matrix(zeros((3,3))) # Matriz de cofactores
  idx=matrix(range(3))
  for i in range(size(A,0)):
      for j in range(size(A,1)):
          fidx=idx[idx!=i]
          cidx=idx[idx!=j]
          cof=A[[[fidx[0,0]],[fidx[0,1]]],cidx]
          MC[i,j]=pow(-1,i+j)*det(cof)
  MADJ = MC.transpose()
  return MADJ


## Definición de variables para el algoritmo de cifrado y descifrado

|Numeros|Caracter|Caracter(10)|Caracter(20)|Caracter(30)|
|------|--------|--------|--------|--------|
| 0 | a | k | t | 3 |
| 1 | b | l | u | 4 |
| 2 | c | m | v | 5 |
| 3 | d | n | w | 6 |
| 4 | e | ñ | x | 7 |
| 5 | f | o | y | 8 |
| 6 | g | p | z | 9 |
| 7 | h | q | 0 |
| 8 | i | r | 1 |
| 9 | j | s | 2 |

In [3]:
#@title Construcción de las tablas de equivalencias
tabla_equivalencias = list(enumerate('abcdefghijklmnñopqrstuvwxyz0123456789'))
letraAnumero_diccionario = dict((letra,numero) for (numero,letra) in tabla_equivalencias)
numeroAletra_diccionario = dict(tabla_equivalencias)
print(letraAnumero_diccionario)
print(numeroAletra_diccionario)

{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'ñ': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '0': 27, '1': 28, '2': 29, '3': 30, '4': 31, '5': 32, '6': 33, '7': 34, '8': 35, '9': 36}
{0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g', 7: 'h', 8: 'i', 9: 'j', 10: 'k', 11: 'l', 12: 'm', 13: 'n', 14: 'ñ', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 27: '0', 28: '1', 29: '2', 30: '3', 31: '4', 32: '5', 33: '6', 34: '7', 35: '8', 36: '9'}


In [4]:
#@title Construcción de la Matriz llave

dimension_de_matriz =3 #@param {type:"integer"}
llave = "kilowatts" #@param {type:"string"}

matriz_llave = [[0 for i in range(dimension_de_matriz)] for i in range(dimension_de_matriz)]

if len(llave) == dimension_de_matriz*dimension_de_matriz:
  matriz_caracteres = hacer_matriz(llave,dimension_de_matriz)
  matriz_llave = transformar_caracteres(matriz_caracteres,letraAnumero_diccionario)
elif len(llave) < dimension_de_matriz*dimension_de_matriz:
  llave_rellenada = rellenar(llave,dimension_de_matriz)
  matriz_caracteres = hacer_matriz(llave_rellenada,dimension_de_matriz)
  matriz_llave = transformar_caracteres(matriz_caracteres,letraAnumero_diccionario)
else:
  print("No se puede recortar la llave")

printm("## Matriz llave $K$:")
print(matriz_llave)
printl(matriz_llave)

## Matriz llave $K$:

[[10, 8, 11], [15, 23, 0], [20, 20, 19]]


<IPython.core.display.Math object>

In [5]:
#@title Construción del mensaje en forma de vectores
from textwrap import wrap

mensaje = "caah970313hdfbgc10" #@param {type:"string"}
longitud_mensaje = len(mensaje)
numero_renglones_vector = 1
lista_de_vectores = []

if longitud_mensaje % dimension_de_matriz == 0 and longitud_mensaje >= dimension_de_matriz:
  lista_de_grupo_de_caracteres = wrap(mensaje,dimension_de_matriz)
  lista_vectores_caracteres = []
  for grupo in lista_de_grupo_de_caracteres:
    lista_vectores_caracteres.append(hacer_matriz(grupo,numero_renglones_vector))
  for vector in lista_vectores_caracteres:
    lista_de_vectores.append(transformar_caracteres(vector,letraAnumero_diccionario))
elif longitud_mensaje >= dimension_de_matriz:
  mensaje_rellenado = rellenar(mensaje,dimension_de_matriz)
  lista_de_grupo_de_caracteres = wrap(mensaje_rellenado,dimension_de_matriz)
  lista_vectores_caracteres = []
  for grupo in lista_de_grupo_de_caracteres:
    lista_vectores_caracteres.append(hacer_matriz(grupo,numero_renglones_vector))
  for vector in lista_vectores_caracteres:
    lista_de_vectores.append(transformar_caracteres(vector,letraAnumero_diccionario))
else:
  print("Este mensaje es muy corto, escribe uno mas largo")

printm("##El mensaje en forma de vectores $M$ queda como :")
for vector in lista_de_vectores:
  printl(vector)




##El mensaje en forma de vectores $M$ queda como :

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Cifrado

$$\left \{
\begin{pmatrix}
 k_{11}  &k_{12}  &... &k_{1n}\\ 
 k_{21}  &k_{22}  &... &k_{2n}\\ 
 :  &:  &... &:\\ 
 k_{n1}  &k_{n2}  &... &k_{nn}
\end{pmatrix} \times
\begin{pmatrix}
 M_{1}  \\ 
 M_{2} \\ 
 : \\ 
 M_{n} 
\end{pmatrix}
\right \} mod N
=\begin{pmatrix}
 C_{1}  \\ 
 C_{2} \\ 
 : \\ 
 C_{n} 
\end{pmatrix}
$$

In [6]:
#@title Cifrado de los vectores
from numpy import array,matmul

vectores_cifrados = []
K = array(matriz_llave)
modulo =  37#@param

for vector in lista_de_vectores:
  M = array(vector)
  vectores_cifrados.append(matmul(K,M)%modulo)

for vectorCifrado in vectores_cifrados:
  printl(vectorCifrado)

for vector in vectores_cifrados:
  print(transformar_caracteres(vector.tolist(),numeroAletra_diccionario))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

[['t'], ['3'], ['d']]
[['2'], ['i'], ['z']]
[['e'], ['v'], ['h']]
[['s'], ['s'], ['t']]
[['n'], ['x'], ['m']]
[['w'], ['i'], ['d']]


## Descifrado

$$M=K^{-1} \times C $$

$$\begin{pmatrix}
 M_{1}  \\ 
 M_{2} \\ 
 : \\ 
 M_{n} 
\end{pmatrix} =
\left \{
\begin{pmatrix}
 k_{11}  &k_{12}  &... &k_{1n}\\ 
 k_{21}  &k_{22}  &... &k_{2n}\\ 
 :  &:  &... &:\\ 
 k_{n1}  &k_{n2}  &... &k_{nn}
\end{pmatrix} \times
\begin{pmatrix}
 C_{1}  \\ 
 C_{2} \\ 
 : \\ 
 C_{n} 
\end{pmatrix} 
\right \} modN
$$

**Donde $K^{-1}$ es la matriz inversa de K modulo N:**
$$
K^{-1}=\left\{
\left(
  \frac{1}{det(K)modulo\mathbf{N}}
\right)
\left(
  adj(K)modulo\mathbf{N}
\right)
\right\}
modulo\mathbf{N}
$$
**Donde $C$ son los vectores del mensaje cifrado**

In [7]:
#@title Cálculo de la matriz inverza de K modulo N
from numpy.linalg import det

determinante_matriz_k = round(det(K))
determinante_matriz_k_modulo_N = determinante_matriz_k%modulo
inverso = inverso_multiplicativo(determinante_matriz_k_modulo_N,modulo)
adj = adjunta(K)
adj_modulo = adj % modulo
matriz_inversa = array((adj_modulo*inverso)%modulo)

printm("**La matriz $K$ es:**")
printl(K)
printm("**El determinante de $K$ es:**" + " " + str(determinante_matriz_k))
printm(f"**El determinante de $K$ modulo {modulo} es:**" + " " + str(determinante_matriz_k_modulo_N))
printm(f"**El inverso multiplicativo del determinante de K modulo {modulo} es:**" + " " + str(inverso))
printm("**La adjunta de $K$ es:**")
printl(adj)
printm(f"**La adjunta de $K$ modulo {modulo} es:**")
printl(adj_modulo)
printm("**La inversa de la matriz es:**")
printl(matriz_inversa.tolist())

**La matriz $K$ es:**

<IPython.core.display.Math object>

**El determinante de $K$ es:** 330

**El determinante de $K$ modulo 37 es:** 34

**El inverso multiplicativo del determinante de K modulo 37 es:** 12

...

<IPython.core.display.Math object>

**La adjunta de $K$ modulo 37 es:**

<IPython.core.display.Math object>

**La inversa de la matriz es:**

<IPython.core.display.Math object>

In [8]:
#@title Descifrado de los vectores
#vectores_cifrados
vectores_descifrados = []

for v in vectores_cifrados:
  vectores_descifrados.append(around(matmul(matriz_inversa,v)) % modulo)

for vector in vectores_descifrados:
  printl(vector)

for vector in vectores_descifrados:
  print(transformar_caracteres(vector.astype(int).tolist(),numeroAletra_diccionario))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

[['c'], ['a'], ['a']]
[['h'], ['9'], ['7']]
[['0'], ['3'], ['1']]
[['3'], ['h'], ['d']]
[['f'], ['b'], ['g']]
[['c'], ['1'], ['0']]
